# Lesson  4 Streamlining Student Interaction with Tutor Controller

Certainly! Here’s your content converted to a well-structured **Markdown** document with code blocks, headings, and clear formatting for readability:

---

# Streamlining Student Interaction with TutorController

Welcome to the next step in our journey of building a personal tutor service with DeepSeek models. In the previous lesson, we explored the `TutorService` class, which acts as a bridge between managing tutoring session data and generating AI responses. Now, we will focus on the **TutorController**, a crucial component that manages tutoring sessions and handles student queries by interacting with both the model and service layers.

The controller is responsible for orchestrating the flow of data between the student interface and the backend services, ensuring that student interactions are processed efficiently and effectively.

---

## Implementing the `TutorController` Class

The `TutorController` class is the heart of our controller layer. It is responsible for managing tutoring sessions and processing student queries.

Let's begin by examining the structure of the `TutorController` class:

```python
import uuid
from services.tutor_service import TutorService

class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}  # Simple session storage for testing
```

**In this snippet, we:**

* Import the `uuid` module from the Python standard library for generating unique identifiers.
* Import the `TutorService` class for managing tutoring data and processing student queries.
* Initialize the `TutorController` with an instance of `TutorService`.
* Create a `test_session` dictionary to simulate session management for testing purposes.

> The `test_session` is a simple dictionary used to simulate session management for testing purposes. In a real-world scenario, student sessions help track individual students as they interact with a web application, maintaining their state and data across multiple requests. By using `test_session`, we can focus on testing the core functionality of the `TutorController` without needing a full session management system. Once we are confident that the controller works correctly, we will integrate a more robust session management solution when developing our RESTful API.

---

## Ensuring Student Session

Before creating a tutoring session, we need to ensure that a student session exists. The `ensure_student_session` method checks if a student ID is present in the `test_session`. If not, it generates a new student ID.

```python
def ensure_student_session(self):
    """Ensure the student has a session ID in the test session."""
    if 'student_id' not in self.test_session:
        self.test_session['student_id'] = str(uuid.uuid4())
    return self.test_session['student_id']
```

This method ensures that a student session is available by checking the `test_session` dictionary for a `student_id`. If it doesn't exist, a new student ID is generated using `uuid.uuid4()` and stored in the session. The method then returns the student ID, either the newly created one or the existing one.

---

## Creating a New Tutoring Session

One of the primary responsibilities of the `TutorController` is to create new tutoring sessions. The `create_session` method handles session creation requests.

```python
def create_session(self):
    """Handle tutoring session creation request."""
    student_id = self.test_session.get('student_id')
    
    if not student_id:
        return self._error_response('Session expired', 401)
    
    session_id = self.tutor_service.create_session(student_id)
    
    return self._success_response({
        'session_id': session_id,
        'message': 'Tutoring session created successfully'
    })
```

**In this method, we:**

* **Retrieve the student\_id:** We first check the `test_session` for a `student_id`.
* **Handle Session Expiry:** If the session has expired (i.e., no `student_id` is found), we return an error response using our helper method `_error_response` with a 401 status code.
* **Create a Tutoring Session:** If the session is valid, we call the `create_session` method of the `TutorService` with the student ID to create a new tutoring session. We then return a success response containing a unique session ID and a success message using our helper method `_success_response`.

---

## Handling Student Queries

The `send_query` method is responsible for processing student queries and returning the tutor's response or an error message.

```python
def send_query(self, session_id=None, student_query=None):
    """Handle tutoring query sending request."""
    student_id = self.test_session.get('student_id')
    if not student_id:
        return self._error_response('Session expired', 401)
    
    if not session_id or not student_query:
        return self._error_response('Missing session_id or query', 400)
        
    try:
        tutor_response = self.tutor_service.process_query(
            student_id, 
            session_id, 
            student_query
        )
        return self._success_response({'message': tutor_response})
        
    except ValueError as e:
        return self._error_response(str(e), 404)
    except RuntimeError as e:
        return self._error_response(str(e), 500)
```

**In this method:**

* We first check if the student session is valid by retrieving the `student_id` from the `test_session`. If no `student_id` is found, we return an error response with a 401 status code.
* We then validate that both `session_id` and `student_query` are provided. If either is missing, we return an error response with a 400 status code.
* If all validations pass, we attempt to process the query using the `process_query` method of the `TutorService`.
* If the query is processed successfully, we return the tutor's response in a success response format.
* If a `ValueError` occurs (e.g., if the session ID doesn't exist), we return an error response with a 404 status code.
* If a `RuntimeError` occurs (e.g., if there's an issue with the AI model), we return an error response with a 500 status code.

---

## Helper Methods for Response Formatting

The `TutorController` includes two helper methods for formatting responses: `_success_response` and `_error_response`. These methods ensure consistent response formats throughout the controller.

```python
def _success_response(self, data):
    response = {
        'status': 'success',
        'data': data
    }
    return response

def _error_response(self, message, status_code):
    response = {
        'status': 'error',
        'error': {
            'message': message,
            'code': status_code
        }
    }
    return response, status_code
```

* `_success_response` returns a dictionary with a status field set to `"success"` and a data field containing the provided data.
* `_error_response` returns a tuple containing a dictionary (with a status field set to `"error"` and an error field with message and code details) and the status code.

---

## Integrating the TutorController in the Main Application

To see the `TutorController` in action, let's integrate it into the main application. This example demonstrates how to create a tutoring session and handle a student query, showcasing the controller's functionality.

```python
from controllers.tutor_controller import TutorController

# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing
student_id = tutor_controller.ensure_student_session()

# Create a new tutoring session
session_response = tutor_controller.create_session()

if isinstance(session_response, tuple) and session_response[0]['status'] == 'error':
    error_data = session_response[0]['error']
    print(f"Error {error_data['code']}: {error_data['message']}")
else:
    session_data = session_response['data']
    session_id = session_data['session_id']
    print(f"Tutoring session created with session_id: {session_id}")

    # Example query handling
    student_query = "What are the key differences between mitosis and meiosis?"

    response = tutor_controller.send_query(session_id, student_query)

    if isinstance(response, tuple) and response[0]['status'] == 'error':
        error_data = response[0]['error']
        print(f"Error {error_data['code']}: {error_data['message']}")
    else:
        print(f"Tutor Response: {response['data']['message']}")
```

In this example:

* We first initialize the `TutorController`.
* We ensure a student session is available for testing.
* We then create a new tutoring session and handle the response.
* If successful, we simulate a student query about the differences between mitosis and meiosis and use the `send_query` method to process it.
* The response is checked for errors, and either the error message or the tutor's response is printed.

This example demonstrates the flow from ensuring a student session to creating a tutoring session and handling a student query, highlighting the controller's role in managing interactions.

---

## Summary and Next Steps

In this lesson, we explored the `TutorController` class and its role in managing tutoring sessions and handling student queries. We learned how to implement the controller, create tutoring sessions, and process student questions using the `TutorService`. The controller is a vital component of our personal tutor application, ensuring that student interactions are managed efficiently and effectively.

As you move on to the practice exercises, take the opportunity to experiment with the `TutorController`'s functionality. This hands-on practice will reinforce the concepts covered in this lesson and prepare you for the next steps in our course.

**Keep up the great work, and I look forward to seeing your progress!**


## Streamlining Student Interaction with Tutor Controller

Your task is to complete the missing parts of the code:

Define the ensure_student_session method in the TutorController class:

Check if the test_session dictionary contains a key 'student_id'.
If 'student_id' is not present, generate a new unique student ID using the uuid module and store it in the test_session dictionary.
Return the student_id from the test_session.
Call this method in the main application to verify its functionality:

Initialize an instance of TutorController.
Use the ensure_student_session method to ensure a student session is created.
Print the student_id to verify that the session has been successfully created.
Dive in and see how this method helps in managing student sessions effectively!

```python
import uuid


class TutorController:
    def __init__(self):
        self.test_session = {}

    # TODO: Define the ensure_student_session method
    # - Check if 'student_id' is in the test_session dictionary
    # - Generate and store a new student_id using uuid if it doesn't exist
    # - Return the student_id from the test_session

from controllers.tutor_controller import TutorController

# TODO: Initialize the TutorController

# TODO: Ensure a student session for testing and print the student ID


```

Here’s the completed code, including the `ensure_student_session` method and how to test it in the main application:

```python
import uuid

class TutorController:
    def __init__(self):
        self.test_session = {}

    def ensure_student_session(self):
        # Check if 'student_id' is in the test_session dictionary
        if 'student_id' not in self.test_session:
            # Generate and store a new student_id using uuid if it doesn't exist
            self.test_session['student_id'] = str(uuid.uuid4())
        # Return the student_id from the test_session
        return self.test_session['student_id']


# Main application
# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing and print the student ID
student_id = tutor_controller.ensure_student_session()
print(f"Student session created with student_id: {student_id}")
```

---

**What this does:**

* Checks for `'student_id'` in the session.
* If missing, generates a new unique ID and stores it.
* Returns (and prints) the student’s session ID.

You can run this snippet as-is to see the result.



## Implementing Student Session Management in TutorController

Great job on understanding the TutorController class so far! Now, let's put that knowledge into practice by focusing on the create_session method. This method is crucial for initiating new tutoring sessions by ensuring each student can start a unique session.

Your task is to complete the missing parts of the code:

Define the create_session method in the TutorController class:

Retrieve the student_id from the test_session dictionary.
If the student_id is not present, return an error response indicating the session has expired.
Use the TutorService to create a new tutoring session with the student_id.
Return a response containing the session_id and a success message.
Call this method in the main application to verify its functionality:

Initialize an instance of TutorController.
Ensure a student session is created using the ensure_student_session method.
Use the create_session method to start a new tutoring session.
Print the session_id to verify that the tutoring session has been successfully created.
Dive in and see how this method helps in managing tutoring sessions effectively!

```python
import uuid
from services.tutor_service import TutorService


class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}

    def ensure_student_session(self):
        """Ensure student has a session ID in the test session."""
        if 'student_id' not in self.test_session:
            self.test_session['student_id'] = str(uuid.uuid4())
        return self.test_session['student_id']
    
    # TODO: Define the create_session method
    # - Retrieve the student_id from the test_session
    # - Return an error if student_id is not present
    # - Use TutorService to create a new tutoring session
    # - Return the session_id and a success message

from controllers.tutor_controller import TutorController

# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing
student_id = tutor_controller.ensure_student_session()

# TODO: Create a new tutoring session and print the session ID

```

Here’s the completed implementation of the `create_session` method in the `TutorController` class, and how to test it in your main application:

```python
import uuid
from services.tutor_service import TutorService

class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}

    def ensure_student_session(self):
        """Ensure student has a session ID in the test session."""
        if 'student_id' not in self.test_session:
            self.test_session['student_id'] = str(uuid.uuid4())
        return self.test_session['student_id']
    
    def create_session(self):
        """Create a new tutoring session for the student."""
        student_id = self.test_session.get('student_id')
        if not student_id:
            # Error response if session expired or student_id missing
            return {"status": "error", "message": "Session expired"}
        
        session_id = self.tutor_service.create_session(student_id)
        return {
            "status": "success",
            "session_id": session_id,
            "message": "Tutoring session created successfully"
        }

# Main application
from controllers.tutor_controller import TutorController

# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing
student_id = tutor_controller.ensure_student_session()

# Create a new tutoring session and print the session ID
response = tutor_controller.create_session()
if response["status"] == "success":
    print(f"Tutoring session created with session_id: {response['session_id']}")
else:
    print(f"Error: {response['message']}")
```

---

**What this does:**

* Implements `create_session` to check for a valid student session, create a tutoring session, and return a structured response.
* In the main application, it prints the new session ID or an error if the session cannot be created.

You can run this code to see the result in action!


## Implementing the Create Session Method in TutorController

You've been doing an excellent job navigating through the TutorController class. Now, let's turn our attention to defining the send_query method, a key component in managing student queries and delivering tutor responses. To implement this method, follow these steps:

Retrieve the student_id from the test_session.
Verify if the student_id is valid; if it's not, return an error message indicating the session has expired with a status code of 401.
Check if both session_id and student_query are provided; if not, return an error message with a status code of 400.
Use the tutor_service to process the student query and get the tutor response.
Return the tutor's response or handle any exceptions that may occur, returning appropriate error messages.
Ensure that HTTP status codes are properly defined and returned for each scenario. For example:
Use 401 for session expiration or unauthorized access,
400 for missing or invalid input,
404 if the session is not found,
500 for unexpected server errors.
Finally, use the TutorController to send a student query and check the response for any errors. If an error exists, print the error message. If there is no error, print the tutor's response.

Dive in and see how this method helps in managing student interactions effectively!

```python
import uuid
from services.tutor_service import TutorService


class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}

    def ensure_student_session(self):
        """Ensure student has a session ID in the test session."""
        if 'student_id' not in self.test_session:
            self.test_session['student_id'] = str(uuid.uuid4())
        return self.test_session['student_id']
    
    def create_session(self):
        """Handle tutoring session creation request."""
        student_id = self.test_session.get('student_id')
        if not student_id:
            return {'error': 'Session expired'}, 401
        
        session_id = self.tutor_service.create_session(student_id)
        return {
            'session_id': session_id,
            'message': 'Tutoring session created successfully'
        }
    
    # TODO: Define the send_query method
        # TODO: Retrieve the student_id from the test_session
        # TODO: Check if the student_id is valid, if not return an error
        # TODO: Validate session_id and student_query
        # TODO: Process the query using the tutor_service
        # TODO: Return the tutor's response or handle exceptions


from controllers.tutor_controller import TutorController

# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing
student_id = tutor_controller.ensure_student_session()

# Create a new tutoring session
session_response = tutor_controller.create_session()

# Handle session creation response
if 'error' in session_response:
    print(f"Error: {session_response['error']}")
else:
    # Extract session_id from the response
    session_id = session_response['session_id']
    
    # Example query handling
    student_query = "Can you explain the concept of inheritance in programming?"

    # TODO: Send the student query and get the response

    # TODO: Check if there is an error in the response
        # TODO: If there is an error, print the error message
    # TODO: If there is no error, print the tutor's response

```

Here’s a **complete implementation** of the `send_query` method and its usage in your main application code, following your requirements:

```python
import uuid
from services.tutor_service import TutorService

class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}

    def ensure_student_session(self):
        """Ensure student has a session ID in the test session."""
        if 'student_id' not in self.test_session:
            self.test_session['student_id'] = str(uuid.uuid4())
        return self.test_session['student_id']
    
    def create_session(self):
        """Handle tutoring session creation request."""
        student_id = self.test_session.get('student_id')
        if not student_id:
            return {'error': 'Session expired'}, 401
        
        session_id = self.tutor_service.create_session(student_id)
        return {
            'session_id': session_id,
            'message': 'Tutoring session created successfully'
        }
    
    def send_query(self, session_id=None, student_query=None):
        """Handle tutoring query sending request."""
        # Retrieve the student_id from the test_session
        student_id = self.test_session.get('student_id')
        # Check if the student_id is valid, if not return an error
        if not student_id:
            return {'error': 'Session expired'}, 401
        # Validate session_id and student_query
        if not session_id or not student_query:
            return {'error': 'Missing session_id or query'}, 400
        try:
            # Process the query using the tutor_service
            tutor_response = self.tutor_service.process_query(
                student_id,
                session_id,
                student_query
            )
            # Return the tutor's response
            return {'message': tutor_response}
        except ValueError as e:
            # Session not found or invalid
            return {'error': str(e)}, 404
        except RuntimeError as e:
            # Unexpected server/model error
            return {'error': str(e)}, 500

# ---------------------------------------
from controllers.tutor_controller import TutorController

# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing
student_id = tutor_controller.ensure_student_session()

# Create a new tutoring session
session_response = tutor_controller.create_session()

# Handle session creation response
if isinstance(session_response, tuple) and 'error' in session_response[0]:
    print(f"Error: {session_response[0]['error']} (Status: {session_response[1]})")
elif 'error' in session_response:
    print(f"Error: {session_response['error']}")
else:
    # Extract session_id from the response
    session_id = session_response['session_id']
    
    # Example query handling
    student_query = "Can you explain the concept of inheritance in programming?"

    # Send the student query and get the response
    query_response = tutor_controller.send_query(session_id, student_query)

    # Check if there is an error in the response
    if isinstance(query_response, tuple) and 'error' in query_response[0]:
        print(f"Error: {query_response[0]['error']} (Status: {query_response[1]})")
    elif 'error' in query_response:
        print(f"Error: {query_response['error']}")
    else:
        print(f"Tutor Response: {query_response['message']}")
```

---

**How it works:**

* `send_query` validates the session and the query, processes it, and handles exceptions with proper status codes.
* The main application handles both success and error cases, printing clear messages based on the returned structure.

You can now **run this script to see error and success handling for student queries in your TutorController**!


## Implementing the send_query Method in TutorController

You've been doing an excellent job navigating through the TutorController class. Now, let's turn our attention to defining the send_query method, a key component in managing student queries and delivering tutor responses. To implement this method, follow these steps:

Retrieve the student_id from the test_session.
Verify if the student_id is valid; if it's not, return an error message indicating the session has expired with a status code of 401.
Check if both session_id and student_query are provided; if not, return an error message with a status code of 400.
Use the tutor_service to process the student query and get the tutor response.
Return the tutor's response or handle any exceptions that may occur, returning appropriate error messages.
Ensure that HTTP status codes are properly defined and returned for each scenario. For example:
Use 401 for session expiration or unauthorized access,
400 for missing or invalid input,
404 if the session is not found,
500 for unexpected server errors.
Finally, use the TutorController to send a student query and check the response for any errors. If an error exists, print the error message. If there is no error, print the tutor's response.

Dive in and see how this method helps in managing student interactions effectively!

```python
import uuid
from services.tutor_service import TutorService


class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}

    def ensure_student_session(self):
        """Ensure student has a session ID in the test session."""
        if 'student_id' not in self.test_session:
            self.test_session['student_id'] = str(uuid.uuid4())
        return self.test_session['student_id']
    
    def create_session(self):
        """Handle tutoring session creation request."""
        student_id = self.test_session.get('student_id')
        if not student_id:
            return {'error': 'Session expired'}, 401
        
        session_id = self.tutor_service.create_session(student_id)
        return {
            'session_id': session_id,
            'message': 'Tutoring session created successfully'
        }
    
    # TODO: Define the send_query method
        # TODO: Retrieve the student_id from the test_session
        # TODO: Check if the student_id is valid, if not return an error
        # TODO: Validate session_id and student_query
        # TODO: Process the query using the tutor_service
        # TODO: Return the tutor's response or handle exceptions


from controllers.tutor_controller import TutorController

# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing
student_id = tutor_controller.ensure_student_session()

# Create a new tutoring session
session_response = tutor_controller.create_session()

# Handle session creation response
if 'error' in session_response:
    print(f"Error: {session_response['error']}")
else:
    # Extract session_id from the response
    session_id = session_response['session_id']
    
    # Example query handling
    student_query = "Can you explain the concept of inheritance in programming?"

    # TODO: Send the student query and get the response

    # TODO: Check if there is an error in the response
        # TODO: If there is an error, print the error message
    # TODO: If there is no error, print the tutor's response

```


Here’s a **complete implementation** of the `send_query` method and its usage in your main application code, following your requirements:

```python
import uuid
from services.tutor_service import TutorService

class TutorController:
    def __init__(self):
        self.tutor_service = TutorService()
        self.test_session = {}

    def ensure_student_session(self):
        """Ensure student has a session ID in the test session."""
        if 'student_id' not in self.test_session:
            self.test_session['student_id'] = str(uuid.uuid4())
        return self.test_session['student_id']
    
    def create_session(self):
        """Handle tutoring session creation request."""
        student_id = self.test_session.get('student_id')
        if not student_id:
            return {'error': 'Session expired'}, 401
        
        session_id = self.tutor_service.create_session(student_id)
        return {
            'session_id': session_id,
            'message': 'Tutoring session created successfully'
        }
    
    def send_query(self, session_id=None, student_query=None):
        """Handle tutoring query sending request."""
        # Retrieve the student_id from the test_session
        student_id = self.test_session.get('student_id')
        # Check if the student_id is valid, if not return an error
        if not student_id:
            return {'error': 'Session expired'}, 401
        # Validate session_id and student_query
        if not session_id or not student_query:
            return {'error': 'Missing session_id or query'}, 400
        try:
            # Process the query using the tutor_service
            tutor_response = self.tutor_service.process_query(
                student_id,
                session_id,
                student_query
            )
            # Return the tutor's response
            return {'message': tutor_response}
        except ValueError as e:
            # Session not found or invalid
            return {'error': str(e)}, 404
        except RuntimeError as e:
            # Unexpected server/model error
            return {'error': str(e)}, 500

# ---------------------------------------
from controllers.tutor_controller import TutorController

# Initialize the TutorController
tutor_controller = TutorController()

# Ensure a student session for testing
student_id = tutor_controller.ensure_student_session()

# Create a new tutoring session
session_response = tutor_controller.create_session()

# Handle session creation response
if isinstance(session_response, tuple) and 'error' in session_response[0]:
    print(f"Error: {session_response[0]['error']} (Status: {session_response[1]})")
elif 'error' in session_response:
    print(f"Error: {session_response['error']}")
else:
    # Extract session_id from the response
    session_id = session_response['session_id']
    
    # Example query handling
    student_query = "Can you explain the concept of inheritance in programming?"

    # Send the student query and get the response
    query_response = tutor_controller.send_query(session_id, student_query)

    # Check if there is an error in the response
    if isinstance(query_response, tuple) and 'error' in query_response[0]:
        print(f"Error: {query_response[0]['error']} (Status: {query_response[1]})")
    elif 'error' in query_response:
        print(f"Error: {query_response['error']}")
    else:
        print(f"Tutor Response: {query_response['message']}")
```

---

**How it works:**

* `send_query` validates the session and the query, processes it, and handles exceptions with proper status codes.
* The main application handles both success and error cases, printing clear messages based on the returned structure.

You can now **run this script to see error and success handling for student queries in your TutorController**!
